In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from primaite.game.session import PrimaiteSession


from primaite import _PRIMAITE_CONFIG, PRIMAITE_PATHS
import logging
_PRIMAITE_CONFIG['log_level']=logging.DEBUG
print(PRIMAITE_PATHS.app_log_dir_path)

In [3]:
import itertools

In [4]:
from primaite.game.session import PrimaiteSession
from primaite.simulator.sim_container import Simulation
from primaite.game.agent.interface import AbstractAgent
from primaite.simulator.network.networks import arcd_uc2_network
import yaml


In [5]:
with open('example_config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)

In [6]:
sess = PrimaiteSession.from_config(cfg)

2023-10-08 17:56:35,831: Added node af2f9c15-ecb4-4b65-b48f-63f12acddb88 to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,836: Added node 47158854-0917-4037-a6a2-33dde56a120f to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,840: Added node cba8ce63-8064-4f80-bcfe-95ca65221dfa to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,846: Added node e01e7c2b-02ac-4e2d-b7bb-8bc3b6ea6509 to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,857: Added node bd5d85ba-5980-45c7-8b28-020a2cfeba0f to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,863: Added node 39e0e37c-4d72-4c76-93cb-4f9c29651ef4 to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,878: Added node 7d1063f9-b5e5-4753-966e-1b630325b266 to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,881: Added node d85b6abb-0f9e-4853-af26-c9b410e1cb94 to Network cbd56fbb-104f-4823-9ee6-f4a968343b31
2023-10-08 17:56:35,884: Added n

service type not found DatabaseBackup
service type not found WebBrowser


In [7]:
sess.agents

In [8]:
network = sess.simulation.network

In [9]:
sess.step()

2023-10-08 17:56:36,041: Stepping primaite session. Step counter: 0
2023-10-08 17:56:36,043: Sending simulation state to agent client_1_green_user
2023-10-08 17:56:36,045: Getting agent action
2023-10-08 17:56:36,047: Formatting agent action DONOTHING
2023-10-08 17:56:36,048: Sending request to simulation: ['do_nothing']
2023-10-08 17:56:36,050: Sending simulation state to agent client_1_data_manipulation_red_bot


[]


TypeError: unhashable type: 'DataManipulationBot'

In [ ]:
from ipaddress import IPv4Address

from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.base import NIC
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.simulator.network.hardware.nodes.router import ACLAction, Router
from primaite.simulator.network.hardware.nodes.server import Server
from primaite.simulator.network.hardware.nodes.switch import Switch
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port
from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.services.database_service import DatabaseService
from primaite.simulator.system.services.dns_client import DNSClient
from primaite.simulator.system.services.dns_server import DNSServer
from primaite.simulator.system.services.red_services.data_manipulation_bot import DataManipulationBot

In [ ]:
router_1 = Router(hostname="router_1", num_ports=5)
router_1.power_on()
router_1.configure_port(port=1, ip_address="192.168.1.1", subnet_mask="255.255.255.0")
router_1.configure_port(port=2, ip_address="192.168.10.1", subnet_mask="255.255.255.0")

# Switch 1
switch_1 = Switch(hostname="switch_1", num_ports=8)
switch_1.power_on()
network.connect(endpoint_a=router_1.ethernet_ports[1], endpoint_b=switch_1.switch_ports[8])
router_1.enable_port(1)

# Switch 2
switch_2 = Switch(hostname="switch_2", num_ports=8)
switch_2.power_on()
network.connect(endpoint_a=router_1.ethernet_ports[2], endpoint_b=switch_2.switch_ports[8])
router_1.enable_port(2)

# Client 1
client_1 = Computer(
    hostname="client_1",
    ip_address="192.168.10.21",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.10.1",
    dns_server=IPv4Address("192.168.1.10"),
)
client_1.power_on()
client_1.software_manager.install(DNSClient)
client_1_dns_client_service: DNSServer = client_1.software_manager.software["DNSClient"]  # noqa
client_1_dns_client_service.start()
network.connect(endpoint_b=client_1.ethernet_port[1], endpoint_a=switch_2.switch_ports[1])
client_1.software_manager.install(DataManipulationBot)
db_manipulation_bot: DataManipulationBot = client_1.software_manager.software["DataManipulationBot"]
db_manipulation_bot.configure(server_ip_address=IPv4Address("192.168.1.14"), payload="DROP TABLE IF EXISTS user;")

# Client 2
client_2 = Computer(
    hostname="client_2",
    ip_address="192.168.10.22",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.10.1",
    dns_server=IPv4Address("192.168.1.10"),
)
client_2.power_on()
client_2.software_manager.install(DNSClient)
client_2_dns_client_service: DNSServer = client_2.software_manager.software["DNSClient"]  # noqa
client_2_dns_client_service.start()
network.connect(endpoint_b=client_2.ethernet_port[1], endpoint_a=switch_2.switch_ports[2])

# Domain Controller
domain_controller = Server(
    hostname="domain_controller",
    ip_address="192.168.1.10",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.1.1",
)
domain_controller.power_on()
domain_controller.software_manager.install(DNSServer)

network.connect(endpoint_b=domain_controller.ethernet_port[1], endpoint_a=switch_1.switch_ports[1])

# Database Server
database_server = Server(
    hostname="database_server",
    ip_address="192.168.1.14",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.1.1",
    dns_server=IPv4Address("192.168.1.10"),
)
database_server.power_on()
network.connect(endpoint_b=database_server.ethernet_port[1], endpoint_a=switch_1.switch_ports[3])

ddl = """
CREATE TABLE IF NOT EXISTS user (
id INTEGER PRIMARY KEY AUTOINCREMENT,
name VARCHAR(50) NOT NULL,
email VARCHAR(50) NOT NULL,
age INT,
city VARCHAR(50),
occupation VARCHAR(50)
);"""

user_insert_statements = [
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('John Doe', 'johndoe@example.com', 32, 'New York', 'Engineer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Jane Smith', 'janesmith@example.com', 27, 'Los Angeles', 'Designer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Bob Johnson', 'bobjohnson@example.com', 45, 'Chicago', 'Manager');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Alice Lee', 'alicelee@example.com', 22, 'San Francisco', 'Student');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('David Kim', 'davidkim@example.com', 38, 'Houston', 'Consultant');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Emily Chen', 'emilychen@example.com', 29, 'Seattle', 'Software Developer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Frank Wang', 'frankwang@example.com', 55, 'New York', 'Entrepreneur');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Grace Park', 'gracepark@example.com', 31, 'Los Angeles', 'Marketing Specialist');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Henry Wu', 'henrywu@example.com', 40, 'Chicago', 'Accountant');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Isabella Kim', 'isabellakim@example.com', 26, 'San Francisco', 'Graphic Designer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Jake Lee', 'jakelee@example.com', 33, 'Houston', 'Sales Manager');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Kelly Chen', 'kellychen@example.com', 28, 'Seattle', 'Web Developer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Lucas Liu', 'lucasliu@example.com', 42, 'New York', 'Lawyer');",  # noqa
    "INSERT INTO user (name, email, age, city, occupation) VALUES ('Maggie Wang', 'maggiewang@example.com', 30, 'Los Angeles', 'Data Analyst');",  # noqa
]
database_server.software_manager.install(DatabaseService)
database_service: DatabaseService = database_server.software_manager.software["DatabaseService"]  # noqa
database_service.start()
database_service._process_sql(ddl, None)  # noqa
for insert_statement in user_insert_statements:
    database_service._process_sql(insert_statement, None)  # noqa

# Web Server
web_server = Server(
    hostname="web_server",
    ip_address="192.168.1.12",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.1.1",
    dns_server=IPv4Address("192.168.1.10"),
)
web_server.power_on()
web_server.software_manager.install(DatabaseClient)

database_client: DatabaseClient = web_server.software_manager.software["DatabaseClient"]
database_client.configure(server_ip_address=IPv4Address("192.168.1.14"))
network.connect(endpoint_b=web_server.ethernet_port[1], endpoint_a=switch_1.switch_ports[2])
database_client.run()
database_client.connect()

# register the web_server to a domain
dns_server_service: DNSServer = domain_controller.software_manager.software["DNSServer"]  # noqa
dns_server_service.start()
dns_server_service.dns_register("arcd.com", web_server.ip_address)

# Backup Server
backup_server = Server(
    hostname="backup_server",
    ip_address="192.168.1.16",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.1.1",
    dns_server=IPv4Address("192.168.1.10"),
)
backup_server.power_on()
network.connect(endpoint_b=backup_server.ethernet_port[1], endpoint_a=switch_1.switch_ports[4])

# Security Suite
security_suite = Server(
    hostname="security_suite",
    ip_address="192.168.1.110",
    subnet_mask="255.255.255.0",
    default_gateway="192.168.1.1",
    dns_server=IPv4Address("192.168.1.10"),
)
security_suite.power_on()
network.connect(endpoint_b=security_suite.ethernet_port[1], endpoint_a=switch_1.switch_ports[7])
security_suite.connect_nic(NIC(ip_address="192.168.10.110", subnet_mask="255.255.255.0"))
network.connect(endpoint_b=security_suite.ethernet_port[2], endpoint_a=switch_2.switch_ports[7])

router_1.acl.add_rule(action=ACLAction.PERMIT, src_port=Port.ARP, dst_port=Port.ARP, position=22)

router_1.acl.add_rule(action=ACLAction.PERMIT, protocol=IPProtocol.ICMP, position=23)

# Allow PostgreSQL requests
router_1.acl.add_rule(
    action=ACLAction.PERMIT, src_port=Port.POSTGRES_SERVER, dst_port=Port.POSTGRES_SERVER, position=0
)

# Allow DNS requests
router_1.acl.add_rule(action=ACLAction.PERMIT, src_port=Port.DNS, dst_port=Port.DNS, position=1)


In [ ]:
node_uuid_list = list(sess.simulation.network.nodes.keys())

In [ ]:
from primaite.game.agent.actions import ActionManager

In [ ]:
actman = ActionManager(sess.simulation, ["DONOTHING", "NODE_SERVICE_SCAN", "NODE_SERVICE_STOP", "NODE_FOLDER_SCAN"],node_uuid_list,act_map={
    0:{
        "action": "DONOTHING",
        "options": {}
    },
    1:{
        "action": "NODE_SERVICE_SCAN",
        "options": {"node_id":0, "service_id":0},
    },
    2:{
        "action": "NODE_SERVICE_SCAN",
        "options": {"node_id":1, "service_id":0},
    },
    3:{
        "action": "NODE_FOLDER_SCAN",
        "options": {"node_id":4, "folder_id":0},
    }
})

In [ ]:
act_id, act_options = actman.get_action(3)
my_trial_act = actman.form_request(action_identifier=act_id, action_options=act_options)

In [ ]:
sess.simulation.apply_action(my_trial_act)

In [ ]:
my_trial_act

In [ ]:
sess.step()

In [ ]:
sess.step_counter

In [ ]:
from gym import spaces

In [ ]:
sp = spaces.Tuple( (spaces.MultiDiscrete([3, 2]), spaces.MultiDiscrete([3, 2]), spaces.MultiDiscrete([3, 2]),))

In [ ]:
sp.sample()

In [ ]:
import yaml
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.simulator.network.hardware.nodes.server import Server
from primaite.simulator.network.hardware.nodes.switch import Switch
from primaite.simulator.network.hardware.nodes.router import Router

from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.services.database_service import DatabaseService
from primaite.simulator.system.services.dns_client import DNSClient
from primaite.simulator.system.services.dns_server import DNSServer
from primaite.simulator.system.services.red_services.data_manipulation_bot import DataManipulationBot


from primaite.simulator.network.hardware.nodes.router import ACLAction
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port

from ipaddress import IPv4Address


In [ ]:
# import yaml


from typing import Dict
from primaite.game.agent.interface import AbstractAgent
from primaite.game.agent.observations import AclObservation, FileObservation, FolderObservation, ICSObservation, LinkObservation, NicObservation, NodeObservation, NullObservation, ServiceObservation, UC2BlueObservation, UC2RedObservation
from primaite.simulator.network.hardware.base import NIC, Link, Node
from primaite.simulator.system.services.service import Service

from primaite.game.agent.scripted_agents import GreenWebBrowsingAgent, RedDatabaseCorruptingAgent
from primaite.game.agent.GATE_agents import GATERLAgent

class PrimaiteSession:

    def __init__(self):
        self.simulation: Simulation
        self.agents = []

    @classmethod
    def from_config(cls, cfg_path):
        ref_map_nodes: Dict[str,Node] = {}
        ref_map_services: Dict[str, Service] = {}
        ref_map_links: Dict[str, Link] = {}
        # ref_map_agents: Dict[str, AgentInterface] = {}


        session = cls()
        with open(cfg_path, 'r') as file:
            conf = yaml.safe_load(file)

        #1. create nodes
        sim = Simulation()
        net = sim.network
        nodes_cfg = conf['simulation']['network']['nodes']
        links_cfg = conf['simulation']['network']['links']
        for node_cfg in nodes_cfg:
            node_ref = node_cfg['ref']
            n_type = node_cfg['type']
            if n_type == 'computer':
                new_node = Computer(hostname = node_cfg['hostname'],
                                    ip_address = node_cfg['ip_address'],
                                    subnet_mask = node_cfg['subnet_mask'],
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg['dns_server'])
            elif n_type == 'server':
                new_node = Server(hostname = node_cfg['hostname'],
                                    ip_address = node_cfg['ip_address'],
                                    subnet_mask = node_cfg['subnet_mask'],
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg.get('dns_server'))
            elif n_type == 'switch':
                new_node = Switch(hostname = node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
            elif n_type == 'router':
                new_node = Router(hostname=node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
                if 'ports' in node_cfg:
                    for port_num, port_cfg in node_cfg['ports'].items():
                        new_node.configure_port(port=port_num,
                                                ip_address=port_cfg['ip_address'],
                                                subnet_mask=port_cfg['subnet_mask'])
                if 'acl' in node_cfg:
                    for r_num, r_cfg in node_cfg['acl'].items():
                        # excuse the uncommon walrus operator ` := `. It's just here as a shorthand, to avoid repeating
                        # this: 'r_cfg.get('src_port')'
                        # Port/IPProtocol. TODO Refactor
                        new_node.acl.add_rule(
                            action = ACLAction[r_cfg['action']],
                            src_port = None if not (p:=r_cfg.get('src_port')) else Port[p],
                            dst_port = None if not (p:=r_cfg.get('dst_port')) else Port[p],
                            protocol = None if not (p:=r_cfg.get('protocol')) else IPProtocol[p],
                            src_ip_address = r_cfg.get('ip_address'),
                            dst_ip_address = r_cfg.get('ip_address'),
                            position = r_num
                        )
            else:
                print('invalid node type')
            if 'services' in node_cfg:
                for service_cfg in node_cfg['services']:
                    service_ref = service_cfg['ref']
                    service_type = service_cfg['type']
                    service_types_mapping = {
                        'DNSClient': DNSClient, # key is equal to the 'name' attr of the service class itself.
                        'DNSServer' : DNSServer,
                        'DatabaseClient': DatabaseClient,
                        'DatabaseService': DatabaseService,
                        # 'database_backup': ,
                        'DataManipulationBot': DataManipulationBot,
                        # 'web_browser'
                    }
                    if service_type in service_types_mapping:
                        new_node.software_manager.install(service_types_mapping[service_type])
                        new_service = new_node.software_manager.software[service_type]
                        ref_map_services[service_ref] = new_service
                    else:
                        print(f"service type not found {service_type}")
                    # service-dependent options
                    if service_type == 'DatabaseClient':
                        if 'options' in service_cfg:
                            opt = service_cfg['options']
                            if 'db_server_ip' in opt:
                                new_service.configure(server_ip_address=IPv4Address(opt['db_server_ip']))
                    if service_type == 'DNSServer':
                        if 'options' in service_cfg:
                            opt = service_cfg['options']
                            if 'domain_mapping' in opt:
                                for domain, ip in opt['domain_mapping'].items():
                                    new_service.dns_register(domain, ip)
            if 'nics' in node_cfg:
                for nic_num, nic_cfg in node_cfg['nics'].items():
                    new_node.connect_nic(NIC(ip_address=nic_cfg['ip_address'], subnet_mask=nic_cfg['subnet_mask']))

            net.add_node(new_node)
            new_node.power_on()
            ref_map_nodes[node_ref] = new_node.uuid

        #2. create links between nodes
        for link_cfg in links_cfg:
            node_a = net.nodes[ref_map_nodes[link_cfg['endpoint_a_ref']]]
            node_b = net.nodes[ref_map_nodes[link_cfg['endpoint_b_ref']]]
            if isinstance(node_a, Switch):
                endpoint_a = node_a.switch_ports[link_cfg['endpoint_a_port']]
            else:
                endpoint_a = node_a.ethernet_port[link_cfg['endpoint_a_port']]
            if isinstance(node_b, Switch):
                endpoint_b = node_b.switch_ports[link_cfg['endpoint_b_port']]
            else:
                endpoint_b = node_b.ethernet_port[link_cfg['endpoint_b_port']]
            new_link = net.connect(endpoint_a=endpoint_a, endpoint_b=endpoint_b)
            ref_map_links[link_cfg['ref']] = new_link.uuid

        session.simulation = sim
        #3. create agents
        game_cfg = conf['game_config']
        ports_cfg = game_cfg['ports']
        protocols_cfg = game_cfg['protocols']
        agents_cfg = game_cfg['agents']

        for agent_cfg in agents_cfg:
            agent_ref = agent_cfg['ref']
            agent_type = agent_cfg['type']
            action_space_cfg = agent_cfg['action_space']
            observation_space_cfg = agent_cfg['observation_space']
            reward_function_cfg = agent_cfg['reward_function']

            # CREATE OBSERVATION SPACE
            if observation_space_cfg is None:
                obs_space = NullObservation()
            elif observation_space_cfg['type'] == 'UC2BlueObservation':
                node_obs_list = []
                link_obs_list = []


                #node ip to index maps ip addresses to node id, as there are potentially multiple nics on a node, there are multiple ip addresses
                node_ip_to_index = {}
                for node_idx, node_cfg in enumerate(nodes_cfg):
                    n_ref = node_cfg['ref']
                    n_obj = net.nodes[ref_map_nodes[n_ref]]
                    for nic_uuid, nic_obj in n_obj.nics.items():
                        node_ip_to_index[nic_obj.ip_address] = node_idx + 2



                for node_obs_cfg in observation_space_cfg['options']['nodes']:
                    node_ref = node_obs_cfg['node_ref']
                    folder_obs_list = []
                    service_obs_list = []
                    if 'services' in node_obs_cfg:
                        for service_obs_cfg in node_obs_cfg['services']:
                            service_obs_list.append(ServiceObservation(where=['network','nodes',ref_map_nodes[node_ref],'services',ref_map_services[service_obs_cfg['service_ref']]]))
                    if 'folders' in node_obs_cfg:
                        for folder_obs_cfg in node_obs_cfg['folders']:
                            file_obs_list = []
                            if 'files' in folder_obs_cfg:
                                for file_obs_cfg in folder_obs_cfg['files']:
                                    file_obs_list.append(FileObservation(where=['network','nodes',ref_map_nodes[node_ref], 'folders',folder_obs_cfg['folder_name'], 'files', file_obs_cfg['file_name']]))
                            folder_obs_list.append(FolderObservation(where=['network','nodes',ref_map_nodes[node_ref], 'folders',folder_obs_cfg['folder_name']], files=file_obs_list))
                    nic_obs_list = []
                    for nic_uuid in net.nodes[ref_map_nodes[node_obs_cfg['node_ref']]].nics.keys():
                        nic_obs_list.append(NicObservation(where=['network','nodes',ref_map_nodes[node_ref],'NICs',nic_uuid]))
                    node_obs_list.append(NodeObservation(where=['network','nodes',ref_map_nodes[node_ref]], services=service_obs_list, folders=folder_obs_list,nics=nic_obs_list, logon_status=False))
                for link_obs_cfg in observation_space_cfg['options']['links']:
                    link_ref = link_obs_cfg['link_ref']
                    link_obs_list.append(LinkObservation(where=['network' ,'links', ref_map_links[link_ref]]))

                acl_obs = AclObservation(node_ip_to_id=node_ip_to_index, ports=game_cfg['ports'], protocols=game_cfg['ports'], where=['network','nodes',observation_space_cfg['options']['acl']['router_node_ref']])
                obs_space = UC2BlueObservation(nodes=node_obs_list,links=link_obs_list,acl=acl_obs, ics=ICSObservation())
            elif observation_space_cfg['type'] == 'UC2RedObservation':
                obs_space = UC2RedObservation.from_config(observation_space_cfg['options'], sim=sim)
            else:
                print("observation space config not specified correctly.")
                obs_space = NullObservation()

            # CREATE ACTION SPACE



            # CREATE REWARD FUNCTION

            # CREATE AGENT
            if agent_type == 'GreenWebBrowsingAgent':
                ...
            elif agent_type == 'GATERLAgent':
                ...
            elif agent_type == 'RedDatabaseCorruptingAgent':
                ...
            else:
                print("agent type not found")


        #4. set up agents' actions and observation spaces.
        return session

s = PrimaiteSession.from_config('example_config.yaml')
# print(s.simulation.describe_state())

In [ ]:
s.agents